In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
import abc
from collections import defaultdict
import json
from itertools import chain, repeat
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
from pathlib import Path
import tensorflow as tf
import tf_utilities as tfu
from tqdm.auto import tqdm
from typing import Iterable, Generator, Optional
import time
import wandb

from dnadb.datasets import Greengenes, Silva
from dnadb import dna, fasta, sample, taxonomy

from deepdna.data.dataset import Dataset
from deepdna.nn.data_generators import SequenceGenerator
from deepdna.nn.models import custom_model, dnabert, load_model, setbert, taxonomy as tax_models
from deepdna.nn.utils import encapsulate_model
from deepdna.nn import layers, functional, utils


In [3]:
tfu.devices.select_gpu(0)

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')],
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

## Dataset

In [4]:
dataset = Dataset("../../Datasets/synthetic")
tax_db_path = list(dataset.taxonomy_dbs(Dataset.Split.Train))[0]
tax_db = taxonomy.TaxonomyDb(tax_db_path)

fasta_path = next(dataset.fasta_dbs(Dataset.Split.Train))
fasta_mapping_path = fasta_path.with_suffix(".mapping.db")
samples = tuple(sample.load_multiplexed_fasta(fasta_path, fasta_mapping_path))
len(samples)

210

In [5]:
tax_db

In [6]:
sample_tax_pairs = list(zip(samples, repeat(tax_db)))

In [7]:
from deepdna.data.samplers import SampleSampler, SequenceSampler
from deepdna.nn.data_generators import _encode_sequences, BatchGenerator
from typing import Any, cast

class SequenceTaxonomyGenerator(BatchGenerator):
    def __init__(
        self,
        fasta_taxonomy_pairs: Iterable[tuple[sample.FastaSample, taxonomy.TaxonomyDb]],
        sequence_length: int,
        taxonomy_id_map: taxonomy.TaxonomyIdMap,
        kmer: int = 1,
        subsample_size: int|None = None,
        batch_size: int = 32,
        batches_per_epoch: int = 100,
        augment_slide: bool = True,
        augment_ambiguous_bases: bool = True,
        balance: bool = False,
        shuffle: bool = True,
        rng: np.random.Generator = np.random.default_rng()
    ):
        super().__init__(
            batch_size=batch_size,
            batches_per_epoch=batches_per_epoch,
            shuffle=shuffle,
            rng=rng
        )
        fasta_samples, taxonomy_dbs = zip(*fasta_taxonomy_pairs)
        self.sample_sampler = SampleSampler(cast(tuple[sample.FastaSample, ...], fasta_samples))
        self.sequence_sampler = SequenceSampler(sequence_length, augment_slide)
        self.taxonomy_dbs: tuple[taxonomy.TaxonomyDb, ...] = cast(Any, taxonomy_dbs)
        self.kmer = kmer
        self.taxonomy_id_map = taxonomy_id_map
        self.subsample_size = subsample_size
        self.augment_ambiguous_bases = augment_ambiguous_bases
        self.balance = balance

    @property
    def sequence_length(self) -> int:
        return self.sequence_sampler.sequence_length

    def generate_batch(
        self,
        rng: np.random.Generator
    ) -> tuple[npt.NDArray[np.int32], npt.NDArray[np.int32]]:
        subsample_size = self.subsample_size or 1
        sequences = np.empty((self.batch_size, subsample_size), dtype=f"<U{self.sequence_length}")
        sample_ids = np.empty(self.batch_size, dtype=np.int32)
        sequence_ids = [None] * self.batch_size
        label_ids = np.empty((self.batch_size, subsample_size), dtype=np.int32)
        samples = self.sample_sampler.sample_with_ids(self.batch_size, self.balance, rng)
        for i, (sample_id, sample) in enumerate(samples):
            tax_db = self.taxonomy_dbs[sample_id]
            sequence_info = tuple(self.sequence_sampler.sample_with_ids(sample, subsample_size, rng))
            sequence_ids[i], sequences[i] = zip(*sequence_info)
            sample_ids[i] = sample_id
            label_ids[i] = [self.taxonomy_id_map[tax_db.fasta_id_to_label(fasta_id)] for fasta_id in sequence_ids[i]]
        sequences = _encode_sequences(sequences, self.augment_ambiguous_bases, self.rng)
        if self.subsample_size is None:
            sequences = np.squeeze(sequences, axis=1)
        sequences = sequences.astype(np.int32)
        if self.kmer > 1:
            sequences = dna.encode_kmers(sequences, self.kmer, not self.augment_ambiguous_bases).astype(np.int32) # type: ignore
        return sample_ids, sequence_ids, sequences, label_ids

    def reduce_batch(self, batch):
        # remove sample IDs and sequence IDs
        return batch[2:]

## Model

In [8]:
# api = wandb.Api()
run = wandb.init(project="setbert-taxonomy-naive", name="64d-150l", id="nk4v4z4t", resume=True)

wandb: Currently logged in as: sirdavidludwig. Use `wandb login --relogin` to force relogin


In [9]:
# path = api.artifact("sirdavidludwig/dnabert-pretrain/dnabert-pretrain-silva-64:v3").download()
path = "../../setbert_pretrain"
setbert_model = load_model(path, setbert.SetBertPretrainModel).base

In [10]:
path = "./logs/dnabert_taxonomy_naive"
dnabert_model = load_model(path, tax_models.NaiveHierarchicalTaxonomyClassificationModel)

In [12]:
encoder = setbert.SetBertEncoderModel(setbert_model, compute_sequence_embeddings=True, output_class=False, output_sequences=True, stop_sequence_embedding_gradient=False)
encoder.chunk_size = None
encoder.base.dnabert_encoder.trainable = True
model = tax_models.NaiveTaxonomyClassificationModel(encoder, dnabert_model.taxonomy_id_map)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4))

In [13]:
encoder.base.dnabert_encoder.trainable_weights[-1]

<tf.Variable 'relative_transformer_block_7/relative_multi_head_attention_7/attention_output/bias:0' shape=(64,) dtype=float32, numpy=
array([ 0.09145296, -0.02899065, -0.0864097 ,  0.027758  ,  0.01339686,
       -0.00845247,  0.00995629, -0.02662934, -0.03677256, -0.02358089,
       -0.03584136,  0.02725827,  0.01927957, -0.00726849,  0.01948893,
        0.0034098 ,  0.00834959, -0.01392625, -0.00409424, -0.09033129,
        0.034411  ,  0.00326619, -0.01773469, -0.0507396 , -0.06094213,
        0.03672664, -0.074256  ,  0.10130385,  0.02195524,  0.05280458,
       -0.02153208, -0.03432152,  0.01011103,  0.08833213, -0.00949545,
       -0.03538151,  0.03269981, -0.00399983, -0.0952319 ,  0.05764344,
       -0.02990684,  0.02238867,  0.00594427,  0.03579369, -0.03227761,
       -0.00528226,  0.04215879,  0.00757783,  0.00958397,  0.02712066,
        0.02522335,  0.01925748,  0.07012557, -0.00175431,  0.02481306,
       -0.02246495,  0.05090724, -0.01450829,  0.00448123, -0.05487642,
  

In [10]:
model = load_model("logs/models/setbert_taxonomy_naive_full")

In [14]:
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1000, 148)]       0         
                                                                 
 set_bert_encoder_model (Set  (None, 1000, 64)         2340672   
 BertEncoderModel)                                               
                                                                 
 dense_51 (Dense)            (None, 1000, 11070)       719550    
                                                                 
Total params: 3,060,222
Trainable params: 3,060,222
Non-trainable params: 0
_________________________________________________________________


In [11]:
common_args = dict(
    sequence_length = 150,
    kmer = 3,
    taxonomy_id_map = model.taxonomy_id_map,
    batch_size = 3,
    subsample_size=1000,
)

train_data = SequenceTaxonomyGenerator(
    sample_tax_pairs,
    batches_per_epoch=100,
    **common_args)
test_data = SequenceTaxonomyGenerator(
    sample_tax_pairs,
    batches_per_epoch=20,
    shuffle=False,
    **common_args) 

In [12]:
wandb_callback = wandb.keras.WandbCallback(save_model=False)
wandb_callback.save_model_as_artifact = False
checkpoint = tf.keras.callbacks.ModelCheckpoint("logs/models/setbert_taxonomy_naive_full", save_best=True)

In [15]:
model.fit(train_data, validation_data=test_data, epochs=550, initial_epoch=450, callbacks=[wandb_callback, checkpoint])

Epoch 451/550
  6/100 [>.............................] - ETA: 2:38 - loss: 0.6357 - sparse_categorical_accuracy: 0.8452WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.8074s vs `on_train_batch_end` time: 0.8810s). Check your callbacks.


100/100 [==============================] - ETA: 0s - loss: 0.6415 - sparse_categorical_accuracy: 0.8437

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets


INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 246s 2s/step - loss: 0.6415 - sparse_categorical_accuracy: 0.8437 - val_loss: 0.5189 - val_sparse_categorical_accuracy: 0.8739
Epoch 452/550
100/100 [==============================] - ETA: 0s - loss: 0.6413 - sparse_categorical_accuracy: 0.8438

...

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 250s 3s/step - loss: 0.5959 - sparse_categorical_accuracy: 0.8540 - val_loss: 0.4773 - val_sparse_categorical_accuracy: 0.8848
Epoch 550/550
100/100 [==============================] - ETA: 0s - loss: 0.5992 - sparse_categorical_accuracy: 0.8536

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets


INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 256s 3s/step - loss: 0.5992 - sparse_categorical_accuracy: 0.8536 - val_loss: 0.4781 - val_sparse_categorical_accuracy: 0.8831


In [16]:
model.fit(train_data, validation_data=test_data, epochs=700, initial_epoch=550, callbacks=[wandb_callback, checkpoint])

Epoch 551/700
  6/100 [>.............................] - ETA: 2:39 - loss: 0.6152 - sparse_categorical_accuracy: 0.8526WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.8101s vs `on_train_batch_end` time: 0.8820s). Check your callbacks.


100/100 [==============================] - ETA: 0s - loss: 0.5917 - sparse_categorical_accuracy: 0.8550

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets


INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 246s 2s/step - loss: 0.5917 - sparse_categorical_accuracy: 0.8550 - val_loss: 0.4760 - val_sparse_categorical_accuracy: 0.8851
Epoch 552/700
100/100 [==============================] - ETA: 0s - loss: 0.5896 - sparse_categorical_accuracy: 0.8555

...

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 302s 3s/step - loss: 0.5439 - sparse_categorical_accuracy: 0.8662 - val_loss: 0.4346 - val_sparse_categorical_accuracy: 0.8944
Epoch 700/700
100/100 [==============================] - ETA: 0s - loss: 0.5454 - sparse_categorical_accuracy: 0.8655

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets


INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 262s 3s/step - loss: 0.5454 - sparse_categorical_accuracy: 0.8655 - val_loss: 0.4343 - val_sparse_categorical_accuracy: 0.8945


In [22]:
model.save("logs/models/setbert_taxonomy_naive_full")

INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets


INFO:tensorflow:Assets written to: logs/models/setbert_taxonomy_naive_full/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
